# Sentence Labeling Dataset Creation

In [1]:
import os
import re

from collections import defaultdict

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from tqdm.notebook import tqdm

from nltk import word_tokenize, sent_tokenize, pos_tag
from nltk.corpus import stopwords
import pymorphy2 as pm2

### 0. Introduction

Создадим обучающую выборку для задачи классификации предложений на возможные/невозможные в генерируемых выжимках по терминам.

Выборка будет состоять из предложений для 10 самых частотных биграмм:

In [2]:
top10_wikigrams = {
    'длина_волна',
    'система_уравнение',
    'магнитный_поле',
    'электрический_поле',
    'момент_время',
    'решение_уравнение',
    'система_координата',
    'решение_задача',
    'комнатный_температура',
    'твёрдый_тело',
}

Впоследствии можно будет взять коллекцию из большего числа биграмм.

### 1. Necessary Preparations

#### 1.1 Loading corpus

Загрузим датасет - корпус научных статей по физике:

In [3]:
data_directory = '../data/elibrary_physics_15k'
article_id = re.compile(r'_([0-9]+).htm$')

file_contents = []
for filename in tqdm(os.scandir(data_directory), desc='Loading data'):
    if filename.is_file():
        file_name = filename.name
        file_id = int(article_id.search(file_name).groups(1)[0])
        
        with open(filename, "r", encoding='utf-8') as f:
            text = f.read()
            file_contents.append((file_id, text))

print(f'Размер корпуса: {len(file_contents)} статей')

Loading data: 0it [00:00, ?it/s]

Размер корпуса: 15000 статей


Удалим из корпуса статьи, написанные не на русском языке.

In [4]:
MIN_CYRILLIC_THRESHOLD = 0.2

cyrillic_sym = re.compile(r'[а-яёА-ЯЁ]')

Алгоритм отсеивания статей по языку:
1. подсчитывается кол-во кириллических символов в статье;
2. если соотношение кириллицы >= MIN_CYRILLIC_THRESHOLD -> статья русскоязычная.

In [5]:
rus_articles = []

for article in tqdm(file_contents, desc='Filtering out non-russian articles'):
    rus_count = 0
    for _ in cyrillic_sym.finditer(article[1]):
        rus_count += 1

    if rus_count >= len(article[1]) * MIN_CYRILLIC_THRESHOLD:
        rus_articles.append(article)

print(f'Удалено нерусскоязычных статей: {len(file_contents) - len(rus_articles)}')

Filtering out non-russian articles:   0%|          | 0/15000 [00:00<?, ?it/s]

Удалено нерусскоязычных статей: 4062


In [6]:
articles_count = len(rus_articles)
print(f'Кол-во русскоязычных статей: {articles_count}')

Кол-во русскоязычных статей: 10938


#### 1.2 Arranging lemmatizing procedures

Напишем вспомогательные функции для анализа лемм:

In [7]:
lemmatizer = pm2.MorphAnalyzer(lang='ru')

pm2_cache = {}
corpus_size = 6546389

In [8]:
def match_bigram(word1_parse, word2_parse):
    """
    Tries to match bigram of w1 and w2 with any of the patterns.
    If succeed, returns a tuple ((w1_norm, w2_norm), pattern).
    """
    word1_pos = word1_parse.tag.POS
    word2_pos = word2_parse.tag.POS

    # failed pattern matching
    if word2_pos != 'NOUN' or word1_pos not in {'NOUN', 'ADJF'}:
        return None

    word1_inflection = (word1_parse.tag.gender, word1_parse.tag.number, word1_parse.tag.case)
    word2_inflection = (word2_parse.tag.gender, word2_parse.tag.number, word2_parse.tag.case)

    # 'adj + noun' pattern
    if (
        word1_pos == 'ADJF' and word2_pos == 'NOUN' 
        and word1_inflection[1] == word2_inflection[1]
        and word1_inflection[2] == word2_inflection[2]
        and (
            word1_inflection[0] == word2_inflection[0]
            or word1_inflection[0] is None and word1_inflection[1] == 'plur'
        )
    ):
        return (word1_parse.normal_form, word2_parse.normal_form), 'прил. + сущ.'

    # 'noun + noun' pattern
    if word1_pos == 'NOUN' and word2_pos == 'NOUN' and word2_inflection[2] == 'gent':
        return (word1_parse.normal_form, word2_parse.normal_form), 'сущ. + сущ.'

    return None

Зададим список стоп-слов:

In [9]:
stop_words = stopwords.words('russian')

stop_words.extend(
    [
        # adjective-like pronouns
        'мой', 'твой', 'ваш', 'наш', 'свой', 'его', 'ее', 'их',
        'тот', 'этот', 'такой', 'таков', 'сей', 'который',
        'весь', 'всякий', 'сам', 'самый', 'каждый', 'любой', 'иной', 'другой',
        'какой', 'каков', 'чей', 'никакой', 'ничей',
        'какой-то', 'какой-либо', 'какой-нибудь', 'некоторый', 'некий',
        # participles
        'соответствующий', 'следующий', 'данный',
        # numerals
        'один',
        # insignificant words
        'друг',
    ]
)

stop_words = set(stop_words)

Зададим список стоп-биграмм:

In [10]:
stop_bigrams = []

stop_bigrams.extend(
    [
        ('крайний', 'мера'), ('сегодняшний', 'день'), ('настоящий', 'время'), ('настоящий', 'работа'),
        ('настоящий', 'статья'), ('точка', 'зрение'), ('первый', 'очередь'), ('последний', 'год'),
    ]
)

stop_bigrams = set(stop_bigrams)

Подходящие слова должны содержать только буквы и дефисы (последние не в начале/конце слова):

In [11]:
def match_unigram(word):
    """Checks if a word is long enough and contains only letters and dashes."""
    if word.startswith('-') or word.endswith('-'):
        return False

    word_dashless = word.replace('-', '')
    return len(word_dashless) >= 3 and word_dashless.isalpha()

Bottleneck лемматизации - обращения к pymorphy2. Сократим кол-во обращений к анализатору, кешируя морфологические разборы токенов:

In [12]:
def get_or_cache_token_parse(lemmatizer, token, cache):
    """Extracts token parse from cache, or adds it to the latter if not found."""
    if token in cache:
        token_parse = cache[token]
    else:
        token_parse = lemmatizer.parse(token)[0]
        cache[token] = token_parse

    return token_parse

Реализуем единый интерфейс сохранения tf для n-грамм:

In [13]:
def update_tf_stats(n_gram_info, n_gram_collection, n):
    """Updates n-gram TF statistics with given n-gram."""
    # unigram
    if n == 1:
        if n_gram_info not in n_gram_collection:
            n_gram_collection[n_gram_info] = {
                'TF': 1,
                'DF': 0,
            }
        else:
            n_gram_collection[n_gram_info]['TF'] += 1
    # bigram
    elif n == 2:
        n_gram, n_gram_pattern = n_gram_info
        if n_gram not in n_gram_collection:
            n_gram_collection[n_gram] = {
                'Articles': [],
                'Pattern': n_gram_pattern,
                'TF': 1,
                'DF': 0,
            }
        else:
            n_gram_collection[n_gram]['TF'] += 1

### 2. Forging Dataset

Сохранять датасет будем под следующим именем:

In [14]:
dataset_full_name = '../data/data_frames/sentence_suitability_for_article.csv'

Датасет вида **[[sentence, related_bigram, suitability], ...]**

#### 2.1 Negative Samples

Для начала сформируем негативную часть выборки - предложения из корпуса elibrary, в которых не встретилось ни одного маркера.

Извлечем слова-маркеры:

In [15]:
markers_file = '../data/relation_markers/markers.txt'

In [16]:
block_markers = {}

with open(markers_file, 'r') as f:
    while (line := f.readline()) != '\n':
        line = line.strip()

        # block headline
        if line[0] == '*' and line[1] != '*':
            cur_block = line[1:]
            block_markers[cur_block] = []
        # block separator
        elif line[0] == '*' and line[1] == '*':
            continue
        # marker word
        else:
            block_markers[cur_block].append(line.lower())

In [17]:
markers_set = []

for markers in block_markers.values():
    markers_set.extend(markers)

markers_set = set(markers_set)

In [18]:
markers_set

{'больше',
 'больший',
 'в настоящее время',
 'вариант',
 'вид',
 'включать',
 'включить',
 'входит в состав',
 'давать возможность',
 'дать возможность',
 'известно',
 'использование',
 'использовать',
 'использоваться',
 'как правило',
 'как принято',
 'компонент',
 'конструкция',
 'меньше',
 'меньший',
 'модуль',
 'недостаток',
 'неэффективность',
 'неэффективный',
 'общеизвестно',
 'общепринятый',
 'обычно',
 'означает',
 'определение',
 'определить',
 'определять',
 'особенность',
 'отличие',
 'отличительная особенность',
 'по большей части',
 'позволить',
 'позволять',
 'похож',
 'предназначить',
 'представлять',
 'преимущество',
 'применение',
 'применить',
 'применять',
 'применяться',
 'принадлежать',
 'разновидность',
 'следует считать',
 'служить',
 'содержат',
 'состав',
 'состоять',
 'сравнение',
 'сравнивать',
 'сравнительный',
 'сравнить',
 'структура',
 'сходство',
 'считается',
 'таким образом',
 'термин',
 'тип',
 'традиционно',
 'традиционный',
 'удобный инструмент',

Загрузим датасет предложений с посчитанными метриками:

In [19]:
bigram_mi3_df = pd.read_csv('../data/data_frames/bigram_tfidf_mi3_scores_full.csv', index_col=0)

Введем пороги, по которым будем отличать длинные предложения от средних и коротких:

In [20]:
SHORT_SENT_THRESHOLD = 0.35
MEDIUM_SENT_THRESHOLD = 0.5
LONG_SENT_THRESHOLD = 0.7

SHORT_SENT_MAX_LEN = 10
MEDIUM_SENT_MAX_LEN = 15

Не будем учитывать предложения без "сказуемого" (без семантического анализа их редко когда можно точно определить):

In [21]:
def has_predicate(lemmatizer, cache, tokens, predicates):
    """Checks if the sentence contains a predicate."""
    for token in tokens:
        token_pos = get_or_cache_token_parse(lemmatizer, token, cache).tag.POS
        if token_pos in predicates:
            return True

    return False

In [22]:
def is_definition(bigram, tokens):
    """Checks if the sentence has a structure <... bigram ... - ...>."""
    hyphens = ('-', '−', '–', '—')
    for x in hyphens:
        if x in tokens:
            split_idx = tokens.index(x)
            pre_define = tokens[:split_idx]

            if bigram[0] in pre_define and bigram[1] in pre_define:
                bigram_end_idx = tokens.index(bigram[1])

                if split_idx - bigram_end_idx <= 5:
                    return True

    return False

Сохраним подходящие контексты:

In [23]:
article_info = dict(rus_articles)

In [24]:
bigram_desc_mi3 = bigram_mi3_df.to_dict('records')
predicates = {'INFN', 'VERB', 'PRTS'}

bigram_contexts = {}
for bigram_info in tqdm(bigram_desc_mi3, desc='Extracting bigram contexts'):
    unigram_1 = bigram_info['unigram_1']
    unigram_2 = bigram_info['unigram_2']
    bigram_name = f'{unigram_1}_{unigram_2}'
    bigram_articles = eval(bigram_info['articles_id_list'])

    # skipping all but top-10 frequent bigrams
    if bigram_name not in top10_wikigrams:
        continue

    matching_sentences = []
    for article_id in bigram_articles:
        sentences = sent_tokenize(article_info[article_id], language='russian')

        for sentence_idx, sentence in enumerate(sentences):
            # sentence must start with a capital letter
            if not sentence[0].isupper():
                continue

            tokens = word_tokenize(sentence.lower(), language='russian')
        
            standardized_tokens = []
            for token in tokens:
                if match_unigram(token):
                    standardized_tokens.append(token)

            for i in range(len(standardized_tokens) - 1):
                token1, token2 = standardized_tokens[i], standardized_tokens[i + 1]
        
                token1_parse = get_or_cache_token_parse(lemmatizer, token1, pm2_cache)
                token2_parse = get_or_cache_token_parse(lemmatizer, token2, pm2_cache)

                # bigram must appear before the predicate
                if token1_parse.tag.POS in predicates or token2_parse.tag.POS in predicates:
                    break
                
                if token1_parse.normal_form in stop_words or token2_parse.normal_form in stop_words:
                    continue
        
                bigram_data = match_bigram(token1_parse, token2_parse)
                if bigram_data is not None:
                    bigram, bigram_pattern = bigram_data
        
                    if bigram in stop_bigrams:
                        continue

                    if bigram == (unigram_1, unigram_2):
                        sent_len = len(standardized_tokens)
                        
                        # sentence must contain a predicate or meet a definition template
                        if (
                            not has_predicate(lemmatizer, pm2_cache, standardized_tokens, predicates)
                            and not is_definition((token1, token2), tokens)
                        ):
                            break

                        # the closer bigram to the beginning of the sentence, the bigger its weight
                        bigram_weight = 1 - i / sent_len
                        
                        # saving only those contexts where bigram appeared in the beginning
                        if (
                            sent_len <= SHORT_SENT_MAX_LEN and bigram_weight >= SHORT_SENT_THRESHOLD
                            or sent_len > SHORT_SENT_MAX_LEN and sent_len <= MEDIUM_SENT_MAX_LEN and bigram_weight >= MEDIUM_SENT_THRESHOLD
                            or sent_len > MEDIUM_SENT_MAX_LEN and bigram_weight >= LONG_SENT_THRESHOLD
                        ):
                            matching_sentences.append(sentence)

                        # bigram found -> stop searching
                        break

    # saving extracted bigram contexts
    if matching_sentences:
        bigram_contexts[bigram_name] = matching_sentences

Extracting bigram contexts:   0%|          | 0/36543 [00:00<?, ?it/s]

Наложим ограничения на попадание в выборку:

In [72]:
def get_suitability_status(sentence, formulae_pattern, good_symbols):
    """Checks all conditions to include the sentence in the dataset."""
    if (
        # no equations
        formulae_pattern.search(sentence) is None
        # no incomplete fragments
        and len(sentence) >= 10
    ):
        bad_symbols = set(sentence.lower()) - good_symbols

        if not bad_symbols:
            # remove the mess in the beginning
            first_letter = re.search(r'[А-ЯA-Z]', sentence)
            if first_letter is not None:
                return first_letter.start()

    return -1

In [63]:
formulae_pattern = re.compile(r'\n.\n.\n')

Отберем все символы, не встречающиеся в формулах:

In [38]:
good_symbols = {
    'а', 'б', 'в', 'г', 'д', 'е', 'ё', 'ж', 'з', 'и', 'й', 'к', 'л', 'м', 'н', 
    'о', 'п', 'р', 'с', 'т', 'у', 'ф', 'х', 'ц', 'ч', 'ш', 'щ', 'ъ', 'ы', 'ь', 'э', 'ю', 'я',
    'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r',
    's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
    '(', ')', '%', '\n', ' ', ',', '.', '"', '\'', '!', '/', ':', ';', '?', '[', ']', '«', '»',
    '³', '—', '–', '−', '…', 
}

In [40]:
# # DEBUG

# bad_symbols = set()

# for filename in tqdm(os.scandir(raw_data_directory), desc='Finding bad symbols'):
#     with open(filename, "r", encoding='utf-8') as f:
#         sentences = sent_tokenize(f.read(), language='russian')

#     for sentence in sentences:
#         bad_symbols |= set(sentence.lower()) - good_symbols

bad_symbols

{'#',
 '*',
 '+',
 '-',
 '<',
 '=',
 '>',
 '\\',
 '^',
 '_',
 '{',
 '|',
 '}',
 '~',
 '\xad',
 '°',
 '±',
 '·',
 '×',
 '́',
 'δ',
 'ε',
 'ζ',
 'θ',
 'λ',
 'π',
 'ρ',
 'φ',
 '“',
 '„',
 '′',
 '\u2061',
 'ℎ',
 '→',
 '↔',
 '∀',
 '∂',
 '∇',
 '∈',
 '∑',
 '∗',
 '∘',
 '∞',
 '∫',
 '∮',
 '≈',
 '≠',
 '⋅',
 '⋯',
 '⟶',
 '⟺',
 '⩽',
 '⩾',
 '�'}

Пройдемся по всем предложениям биграмм и классифицируем их по наличию слов-маркеров:

In [65]:
negative_samples = []

for bigram, contexts in tqdm(bigram_contexts.items(), desc='Filling dataset with negative samples'):
    for sentence in contexts:
        is_matching = True

        for marker in markers_set:
            raw_sentence = sentence.lower()
            
            if raw_sentence.find(marker) != -1:
                is_matching = False
                break

        if is_matching:
            sentence_start = get_suitability_status(sentence, formulae_pattern, good_symbols)
            if sentence_start != -1:
                negative_samples.append([sentence[sentence_start:], bigram, 0])

Filling dataset with negative samples:   0%|          | 0/10 [00:00<?, ?it/s]

In [66]:
len(negative_samples)

2910

#### 2.2 Positive Samples

Соберем позитивную часть выборки - предложения из эталонных статей соответствующих биграмм на википедии.

In [27]:
raw_data_directory = '../data/top10_wikigrams/wiki_data_raw'

In [67]:
positive_samples = []

for filename in tqdm(os.scandir(raw_data_directory), desc='Scanning raw data'):
    with open(filename, "r", encoding='utf-8') as f:
        sentences = sent_tokenize(f.read(), language='russian')

    bigram = filename.name[:-4]
    for sentence in sentences:
        sentence_start = get_suitability_status(sentence, formulae_pattern, good_symbols)
        if sentence_start != -1:
            positive_samples.append([sentence[sentence_start:], bigram, 1])

Scanning raw data: 0it [00:00, ?it/s]

In [68]:
len(positive_samples)

548

#### 2.3 Merging Parts

Соединим негативные и позитивные примеры в 1 датасет:

In [69]:
dataset = negative_samples + positive_samples

ds_df = pd.DataFrame(dataset, columns=['text', 'related bigram', 'suitability'])

In [70]:
ds_df

,text,related bigram,suitability
0,Для нитрата никеля(II) зависимости изменения о...,длина_волна,0
1,График зависимости инкремента от длины волны и...,длина_волна,0
2,Изменение отражения для каждой длины волны опр...,длина_волна,0
3,На рис. 1а полоса поглощения на длине волны 58...,длина_волна,0
4,Детальнее ход этих зависимостей для поглощения...,длина_волна,0
...,...,...,...
3453,Основными средствами коллективной защиты от во...,электрический_поле,1
3454,Переносные и передвижные экранирующие устройст...,электрический_поле,1
3455,"В заземлённых кабинах и кузовах машин, механиз...",электрический_поле,1
3456,This aura is the electric field due to the cha...,электрический_поле,1


In [71]:
ds_df.to_csv(dataset_full_name)

### 3. Conclusion

#### 3.1 Notes

- чистка от формул **нужна была в обеих частях** выборки;
- выборка **сократилась почти в 2 раза** (пропорция 1 к 6 сохранилась), но некоторые формулы остались. Я убрал большую часть, остались безобидные по типу "...область, в которой интенсивность выше порога многофотонной z fll...". Т.е. это формулы, почти целиком состоящие из латинских букв. Удалять их не стал по 2 причинам: латинницей обозначаются физические величины, а они сплошь и рядом (получится слишком жесткая чистка), и в корпусе есть полноценные предложения на английском языке (их мало, но кто знает, останется ли это так при расширении выборки);
- встречались хорошие, законченные предложения, **в начале которых была библиографическая информация** (ссылки), такие предварительно очищал от этой ненужной части;
- к слову о библиографии: ссылки вида "[1]" **встречаются и внутри предложений**. Стоит ли регуляркой пройтись по всему корпусу, и удалить подобные фрагменты? Чтобы у моделей возникало минимум трудностей при дообучении.

#### 3.2 Improvements

- **TO-DO:**
    1. ;
    2. ;
    3. .